In [26]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import csv
import json

Convert txt file into csv file

In [24]:
file_path = 'data/rnaseq_tsv/gdc_manifest.2024-02-01.txt'
output_csv_path = 'data/rnaseq_csv/gdc_manifest.2024-02-01.csv'

with open(file_path, 'r') as infile, open(output_csv_path, 'w') as outfile:
    lines = infile.readlines()
    
    for line in lines:
        outfile.write(','.join(line.split('\t')))


Add case id

In [20]:
csv_path = 'data/rnaseq_csv/gdc_manifest.2024-02-01.csv'
df = pd.read_csv(csv_path)

df['caseid'] = df['filename'].str.extract(r'(TCGA-[A-Z0-9-]+)')

output_csv_path = 'data/rnaseq_csv/gdc_manifest_with_caseid.2024-02-01.csv'
df.to_csv(output_csv_path, index=False)

Convert tsv file into csv file

In [19]:
folder_path = 'data/rnaseq_tsv'
output_folder_path = 'data/rnaseq_csv'


for filename in os.listdir(folder_path):
    if filename.endswith('.tsv'):
        tsv_file_path = os.path.join(folder_path, filename)
        csv_file_path = os.path.join(output_folder_path, filename.replace('.tsv', '.csv'))

        with open(tsv_file_path, 'r') as infile, open(csv_file_path, 'w', newline='') as outfile:
            tsv_reader = csv.reader(infile, delimiter='\t')
            csv_writer = csv.writer(outfile)

            for row in tsv_reader:
                csv_writer.writerow(row)
        
        print(f'Converted {tsv_file_path} to {csv_file_path}')

Converted data/rnaseq_tsv\rnaseq_all.tsv to data/rnaseq_csv\rnaseq_all.csv
Converted data/rnaseq_tsv\rnaseq_s1.tsv to data/rnaseq_csv\rnaseq_s1.csv
Converted data/rnaseq_tsv\rnaseq_s1_s2.tsv to data/rnaseq_csv\rnaseq_s1_s2.csv
Converted data/rnaseq_tsv\rnaseq_s2.tsv to data/rnaseq_csv\rnaseq_s2.csv


Add caseid to each file

In [23]:
# Paths to the CSV files
rnaseq_csv_path = 'data/rnaseq_csv/rnaseq_s1_s2.csv'  # Update this to the correct path
manifest_csv_path = 'data/rnaseq_csv/gdc_manifest_with_caseid.2024-02-01.csv'  # Update this to the correct path

# Load the rnaseq and manifest data into pandas dataframes
rnaseq_df = pd.read_csv(rnaseq_csv_path)
manifest_df = pd.read_csv(manifest_csv_path)

# Extract caseid from the filename in the manifest dataframe
manifest_df['caseid'] = manifest_df['filename'].str.extract(r'(TCGA-[A-Z0-9-]+)')

# Create a dictionary mapping from id to caseid
id_to_caseid = manifest_df.set_index('id')['caseid'].to_dict()

# Map the file_id to caseid in rnaseq_df and insert it as the first column
rnaseq_df.insert(0, 'caseid', rnaseq_df['file_id'].map(id_to_caseid))

# Save the updated DataFrame to a new CSV file
updated_rnaseq_csv_path = 'path/to/your/updated_rnaseq_s1_s2.csv'  # Update to your path
# Save the updated DataFrame to a new CSV file
rnaseq_df.to_csv('data/processed_rnaseq_csv/updated_rnaseq_s1_s2.csv', index=False)

Add case id to rnaseq data

In [22]:
with open('data/metadata.cart.2024-02-01.json', 'r') as file:
    json_data = json.load(file)

file_id_to_entity_submitter_id = {}
for item in json_data:
    if "file_id" in item and "associated_entities" in item and len(item["associated_entities"]) > 0:
        file_id = item["file_id"]
        entity_submitter_id = item["associated_entities"][0].get("entity_submitter_id", "")
        if entity_submitter_id:
            parts = entity_submitter_id.split("-")
            if len(parts) >= 3:
                extracted_id = "-".join(parts[:3])
                file_id_to_entity_submitter_id[file_id] = extracted_id

df = pd.read_csv('data/rnaseq_csv/rnaseq_all.csv')

df['case_id'] = df['file_id'].map(file_id_to_entity_submitter_id).fillna('Not Found')

cols = ['case_id'] + [col for col in df if col != 'case_id']
df = df[cols]

df.to_csv('data/processed_rnaseq_csv/updated_rnaseq_all.csv', index=False)

Find if there is duplicate case in rnaseq_s1/rnaseq_s1_s2/rnaseq_all

In [32]:

df = pd.read_csv('data/processed_rnaseq_csv/updated_rnaseq_s1.csv') 

duplicated_case_ids = df[df.duplicated('case_id', keep=False)]

unique_case_ids = df.drop_duplicates('case_id', keep=False)

unique_case_ids_array = unique_case_ids['case_id'].tolist()
duplicated_case_ids_array = duplicated_case_ids['case_id'].tolist()

print(f"non-Duplicate number: {len(unique_case_ids_array)}")
print(f"Duplicate number: {len(duplicated_case_ids_array)}")
print(duplicated_case_ids_array)

非重复的case_id数量: 475
['TCGA-CV-7183', 'TCGA-CV-6933', 'TCGA-CV-7178', 'TCGA-CV-6959', 'TCGA-CV-7425', 'TCGA-CV-7252', 'TCGA-CV-6956', 'TCGA-CV-7438', 'TCGA-CV-6943', 'TCGA-CV-7255', 'TCGA-CV-7425', 'TCGA-CV-6961', 'TCGA-CV-7242', 'TCGA-CV-7101', 'TCGA-CV-7235', 'TCGA-KU-A6H7', 'TCGA-CV-6938', 'TCGA-CV-7255', 'TCGA-CV-6955', 'TCGA-CV-7101', 'TCGA-HD-A6I0', 'TCGA-CV-7261', 'TCGA-HD-8635', 'TCGA-CV-6934', 'TCGA-CV-6936', 'TCGA-HD-8635', 'TCGA-CV-7406', 'TCGA-H7-A6C4', 'TCGA-CV-7434', 'TCGA-CV-7178', 'TCGA-CV-7252', 'TCGA-CV-7434', 'TCGA-HD-A6I0', 'TCGA-CV-7235', 'TCGA-CV-7183', 'TCGA-CV-6933', 'TCGA-CV-6938', 'TCGA-CV-7416', 'TCGA-HD-A6HZ', 'TCGA-CV-7245', 'TCGA-CV-6961', 'TCGA-CV-7406', 'TCGA-CV-6934', 'TCGA-KU-A6H7', 'TCGA-HD-A6HZ', 'TCGA-CV-6962', 'TCGA-CV-7437', 'TCGA-CV-7440', 'TCGA-CV-7238', 'TCGA-CV-7424', 'TCGA-CV-7438', 'TCGA-CV-6939', 'TCGA-CV-6943', 'TCGA-CV-7250', 'TCGA-CV-6959', 'TCGA-CV-7103', 'TCGA-CV-7423', 'TCGA-CV-7250', 'TCGA-WA-A7GZ', 'TCGA-CV-7177', 'TCGA-CV-7097', 'TCG

In [40]:
with open('data/biospecimen.cart.2024-02-01.json', 'r') as file:
    data_list = json.load(file)

# 准备写入CSV
with open('data/processed_rnaseq_csv/rnaseq_biospecimen_results.csv', 'w', newline='') as csvfile:
    fieldnames = ['case_id', 'TCGA_code', 'State', 'aliquot_id']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for data in data_list:
        case_id = data['case_id']

        for sample in data['samples']:
            state = sample['sample_type']  # 'Primary Tumor' 或 'Metastatic'
            last_aliquot_id = None
            TCGA_code = None

            for portion in sample['portions']:
                for analyte in portion.get('analytes', []):
                    if analyte['analyte_type_id'] == "R":  # 确保是RNA测序数据
                        for aliquot in analyte['aliquots']:
                            last_aliquot_id = aliquot['aliquot_id']  # 更新为最后一个aliquot_id
                            submitter_id = aliquot['submitter_id']
                            TCGA_code = '-'.join(submitter_id.split('-')[:3])
            
            if last_aliquot_id:  # 如果找到了RNA测序的aliquot_id，写入CSV
                writer.writerow({'case_id': case_id, 'TCGA_code': TCGA_code, 'State': state, 'aliquot_id': last_aliquot_id})

FileNotFoundError: [Errno 2] No such file or directory: 'your_data_file.json'